In [74]:
import statsmodels.formula.api as smf
import pandas as pd
import numpy as np
import os

In [2]:
def create_regressor_columns_string(columns):
  # Join all independent variables to define the formula used by the model
  regressor_columns = list(filter(lambda x: x != 'impression_count',columns))
  regressor_columns_string = "+".join(regressor_columns)
  return regressor_columns_string

In [121]:
def perform_regression(regression_df, significant_variables):
    # Define formula without cross terms
    regressor_columns_string = create_regressor_columns_string(significant_variables)
    # Fit model
    mod = smf.ols(formula=f'impression_count ~ {regressor_columns_string}', data=regression_df)
    res = mod.fit()
    return res

In [122]:
def keep_significant_var(p_values):
    # Filter p-values < 0.05
    p_values = p_values[p_values < 0.05]

    # Get significant variables
    significant_variables = list(p_values.index)
    # Remove intercept if significant
    try:
        significant_variables.remove('Intercept')
    except:
        pass

    return significant_variables

In [138]:
def regression(path, threshold=1e2):
    # Load data
    regression_df_pd = pd.read_csv(path)
    regression_df_pd = regression_df_pd.drop('tweet_text', axis=1)
    
    #======================#
    #   FIRST REGRESSION   #
    #======================#
    significant_variables = list(regression_df_pd.columns)
    res = perform_regression(regression_df_pd, significant_variables)
    p_values = res.pvalues
    nb_variables_before = len(p_values)
    
    #======================#
    #   SECOND REGRESSION  #
    #======================#
    # Filter out data if impression_count < threshold
    regression_df_pd = regression_df_pd[regression_df_pd['impression_count'] >= threshold].copy()

    # Perform new regression with significant variables
    res_threshold = perform_regression(regression_df_pd, significant_variables)

    #======================#
    #   THIRD REGRESSION   #
    #======================#
    # Apply log transformation to independent variables
    for var in regression_df_pd.columns:
            regression_df_pd[var] = regression_df_pd[var].apply(lambda x: np.log(1+x))

    # Perform new regression with significant variables
    res_log = perform_regression(regression_df_pd, significant_variables)
    p_values = res_log.pvalues

    #======================#
    #   FINAL RESULTS      #
    #======================#
    # Filter p-values < 0.05
    significant_variables = keep_significant_var(p_values)
    nb_variables_after = len(significant_variables)

    # Perform new regression with significant variables
    res_final = perform_regression(regression_df_pd, significant_variables)

    print(f'Number of discarded variables: {nb_variables_before - nb_variables_after}')
    print(f'Significant variables ({len(significant_variables)}): {significant_variables}')

    #======================#
    #   SAVE AS HTML       #
    #======================#
    # Save first res as html
    res_html = res.summary().as_html()
    name = path.split('/')[-1].split('.')[0][:-3]
    path_root = os.path.join(os.getcwd(), "data", "regression", "html_regression")
    path = os.path.join(path_root, name+".html")
    with open(path, 'w') as f:
        f.write(res_html)

    # Save second res as html
    res_thresh_html = res_threshold.summary().as_html()
    path_opti = os.path.join(path_root, name+"_threshold.html")
    with open(path_opti, 'w') as f:
        f.write(res_thresh_html)

    # Save third res as html
    res_log_html = res_log.summary().as_html()
    path_log = os.path.join(path_root, name+"_log.html")
    with open(path_log, 'w') as f:
        f.write(res_log_html)

    # Save final res as html
    res_final_html = res_final.summary().as_html()
    path_final = os.path.join(path_root, name+"_final.html")
    with open(path_final, 'w') as f:
        f.write(res_final_html)

    return res, res_threshold, res_log, res_final

In [139]:
path_french_celebrities = '../data/regression/french_celebrities_regression_df.csv'
res_french_celebrities = regression(path_french_celebrities)

display(res_french_celebrities[0].summary())
display(res_french_celebrities[1].summary())
display(res_french_celebrities[2].summary())
display(res_french_celebrities[3].summary())

Number of discarded variables: 9
Significant variables (15): ['dummy_entertainmentleisurebusiness', 'dummy_jeanlucmlenchon', 'dummy_nikosaliagas', 'dummy_politicalfigures', 'dummy_soccer', 'dummy_sportsfitnessbusiness', 'dummy_tweet_period_afternoon', 'dummy_tweet_period_morning', 'dummy_tweet_period_night', 'hashtags_count', 'mentions_count', 'tweet_external_urls_count', 'tweet_length', 'tweet_medias_count', 'followers_count']


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       impression_count   R-squared:                       0.183
Model:                            OLS   Adj. R-squared:                  0.182
Method:                 Least Squares   F-statistic:                     172.6
Date:                Sun, 14 May 2023   Prob (F-statistic):               0.00
Time:                        17:29:48   Log-Likelihood:            -2.2180e+05
No. Observations:               16151   AIC:                         4.437e+05
Df Residuals:                   16129   BIC:                         4.438e+05
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
======================================================================================================
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Intercept                           -1.71e+04   3213.965     -5.321      0.000   -2.34e+04   -1.08e+04
dummy_celebrities                    1.24e+04   2.89e+04      0.429      0.668   -4.42e+04     6.9e+04
dummy_entertainment                -3.593e+04   8510.562     -4.222      0.000   -5.26e+04   -1.92e+04
dummy_entertainmentleisurebusiness  2.427e+04   7351.944      3.302      0.001    9862.635    3.87e+04
dummy_francepolitics                8.179e+04   1.19e+04      6.879      0.000    5.85e+04    1.05e+05
dummy_jeanlucmlenchon              -1.865e+05   1.03e+04    -18.111      0.000   -2.07e+05   -1.66e+05
dummy_nikosaliagas                 -1.017e+05   7631.240    -13.326      0.000   -1.17e+05   -8.67e+04
dummy_politicalfigures             -6093.0430   9699.898     -0.628      0.530   -2.51e+04    1.29e+04
dummy_soccer                        1.282e+05   2.17e+04      5.903      0.000    8.57e+04    1.71e+05
dummy_sports                       -1939.6258   2.38e+04     -0.081      0.935   -4.86e+04    4.48e+04
dummy_sportsfitnessbusiness        -1.673e+04   2.39e+04     -0.700      0.484   -6.36e+04    3.01e+04
dummy_staderennaisfc               -1.003e+05   1.95e+04     -5.140      0.000   -1.39e+05    -6.2e+04
dummy_tvhosts                       -4.19e+04    1.5e+04     -2.800      0.005   -7.12e+04   -1.26e+04
dummy_tvstars                       -4.19e+04    1.5e+04     -2.800      0.005   -7.12e+04   -1.26e+04
dummy_tweet_period_afternoon       -6287.0239   2756.049     -2.281      0.023   -1.17e+04    -884.862
dummy_tweet_period_morning         -6410.7703   2920.076     -2.195      0.028   -1.21e+04    -687.096
dummy_tweet_period_night           -4402.9105   2581.026     -1.706      0.088   -9462.009     656.188
hashtags_count                     -3167.7802   2097.988     -1.510      0.131   -7280.070     944.510
mentions_count                     -4645.4856   1025.391     -4.530      0.000   -6655.367   -2635.604
tweet_external_urls_count           5687.9314   4624.156      1.230      0.219   -3375.928    1.48e+04
tweet_length                          57.6515     27.365      2.107      0.035       4.012     111.291
tweet_medias_count                  1.254e+04   3226.829      3.887      0.000    6217.351    1.89e+04
tweet_sentiment                     1878.6859   5351.569      0.351      0.726   -8610.983    1.24e+04
followers_count                        0.0785      0.001     53.046      0.000       0.076       0.081
==============================================================================
Omnibus:                    42710.957   Durbin-Watson:                   1.413
Prob(Omnibus):                  0.000   Jarque-Bera (JB):       1471275351.624
Skew:                          31.333   Prob(JB):                

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       impression_count   R-squared:                       0.256
Model:                            OLS   Adj. R-squared:                  0.253
Method:                 Least Squares   F-statistic:                     116.5
Date:                Sun, 14 May 2023   Prob (F-statistic):               0.00
Time:                        17:29:48   Log-Likelihood:            -1.0073e+05
No. Observations:                7151   AIC:                         2.015e+05
Df Residuals:                    7129   BIC:                         2.017e+05
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
======================================================================================================
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Intercept                          -1794.5796   6871.938     -0.261      0.794   -1.53e+04    1.17e+04
dummy_celebrities                   6406.9282   7.13e+04      0.090      0.928   -1.33e+05    1.46e+05
dummy_entertainment                -2.613e+04   2.06e+04     -1.268      0.205   -6.65e+04    1.43e+04
dummy_entertainmentleisurebusiness  3.112e+04    1.8e+04      1.724      0.085   -4260.464    6.65e+04
dummy_francepolitics                7.381e+04   2.43e+04      3.033      0.002    2.61e+04    1.22e+05
dummy_jeanlucmlenchon              -2.003e+05   2.01e+04     -9.952      0.000    -2.4e+05   -1.61e+05
dummy_nikosaliagas                 -1.658e+05   1.94e+04     -8.542      0.000   -2.04e+05   -1.28e+05
dummy_politicalfigures              3800.6064   2.03e+04      0.187      0.852    -3.6e+04    4.36e+04
dummy_soccer                        2.278e+05   4.49e+04      5.075      0.000     1.4e+05    3.16e+05
dummy_sports                        8317.9641   3.95e+04      0.211      0.833   -6.91e+04    8.57e+04
dummy_sportsfitnessbusiness        -1.702e+04    4.2e+04     -0.405      0.685   -9.93e+04    6.53e+04
dummy_staderennaisfc                -2.31e+05   3.97e+04     -5.820      0.000   -3.09e+05   -1.53e+05
dummy_tvhosts                      -8.916e+04   3.67e+04     -2.429      0.015   -1.61e+05   -1.72e+04
dummy_tvstars                      -8.916e+04   3.67e+04     -2.429      0.015   -1.61e+05   -1.72e+04
dummy_tweet_period_afternoon       -4574.0829   5918.616     -0.773      0.440   -1.62e+04    7028.162
dummy_tweet_period_morning          2467.6584   6283.314      0.393      0.695   -9849.502    1.48e+04
dummy_tweet_period_night             311.8448   5571.112      0.056      0.955   -1.06e+04    1.12e+04
hashtags_count                      3088.8252   3990.250      0.774      0.439   -4733.249    1.09e+04
mentions_count                     -3182.6798   2070.543     -1.537      0.124   -7241.559     876.199
tweet_external_urls_count           -1.84e+04   8470.962     -2.173      0.030    -3.5e+04   -1799.344
tweet_length                         -18.6366     46.733     -0.399      0.690    -110.248      72.975
tweet_medias_count                 -8072.2277   5753.634     -1.403      0.161   -1.94e+04    3206.602
tweet_sentiment                    -1.177e+04   1.13e+04     -1.043      0.297   -3.39e+04    1.04e+04
followers_count                        0.1150      0.003     42.674      0.000       0.110       0.120
==============================================================================
Omnibus:                    16283.414   Durbin-Watson:                   1.512
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        149286541.979
Skew:                          21.629   Prob(JB):                

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       impression_count   R-squared:                       0.615
Model:                            OLS   Adj. R-squared:                  0.613
Method:                 Least Squares   F-statistic:                     541.1
Date:                Sun, 14 May 2023   Prob (F-statistic):               0.00
Time:                        17:29:48   Log-Likelihood:                -11870.
No. Observations:                7151   AIC:                         2.378e+04
Df Residuals:                    7129   BIC:                         2.394e+04
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
======================================================================================================
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Intercept                             -3.3186      0.178    -18.680      0.000      -3.667      -2.970
dummy_celebrities                     -0.1320      0.413     -0.320      0.749      -0.941       0.677
dummy_entertainment                    0.1190      0.120      0.995      0.320      -0.115       0.354
dummy_entertainmentleisurebusiness     0.7316      0.105      6.991      0.000       0.526       0.937
dummy_francepolitics                   0.0567      0.138      0.409      0.682      -0.215       0.328
dummy_jeanlucmlenchon                 -0.7481      0.124     -6.047      0.000      -0.991      -0.506
dummy_nikosaliagas                    -1.9281      0.123    -15.739      0.000      -2.168      -1.688
dummy_politicalfigures                 0.6004      0.118      5.093      0.000       0.369       0.831
dummy_soccer                           0.7371      0.259      2.848      0.004       0.230       1.244
dummy_sports                          -0.2495      0.229     -1.092      0.275      -0.697       0.198
dummy_sportsfitnessbusiness            0.5028      0.243      2.067      0.039       0.026       0.980
dummy_staderennaisfc                  -0.0655      0.228     -0.287      0.774      -0.513       0.382
dummy_tvhosts                         -0.1077      0.214     -0.503      0.615      -0.527       0.312
dummy_tvstars                         -0.1077      0.214     -0.503      0.615      -0.527       0.312
dummy_tweet_period_afternoon          -0.7654      0.052    -14.637      0.000      -0.868      -0.663
dummy_tweet_period_morning            -0.7730      0.054    -14.355      0.000      -0.879      -0.667
dummy_tweet_period_night              -0.7619      0.049    -15.450      0.000      -0.859      -0.665
hashtags_count                         0.1851      0.036      5.098      0.000       0.114       0.256
mentions_count                        -0.9183      0.030    -30.577      0.000      -0.977      -0.859
tweet_external_urls_count              0.8452      0.054     15.772      0.000       0.740       0.950
tweet_length                           0.4771      0.023     21.133      0.000       0.433       0.521
tweet_medias_count                     0.6790      0.045     15.205      0.000       0.591       0.767
tweet_sentiment                       -0.0566      0.042     -1.337      0.181      -0.139       0.026
followers_count                        0.8217      0.015     54.855      0.000       0.792       0.851
==============================================================================
Omnibus:                      179.116   Durbin-Watson:                   1.416
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              294.213
Skew:                           0.232   Prob(JB):                

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       impression_count   R-squared:                       0.614
Model:                            OLS   Adj. R-squared:                  0.613
Method:                 Least Squares   F-statistic:                     810.0
Date:                Sun, 14 May 2023   Prob (F-statistic):               0.00
Time:                        17:29:48   Log-Likelihood:                -11877.
No. Observations:                7151   AIC:                         2.378e+04
Df Residuals:                    7136   BIC:                         2.389e+04
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
======================================================================================================
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Intercept                             -3.1673      0.165    -19.152      0.000      -3.491      -2.843
dummy_entertainmentleisurebusiness     0.7801      0.085      9.200      0.000       0.614       0.946
dummy_jeanlucmlenchon                 -0.6761      0.115     -5.881      0.000      -0.901      -0.451
dummy_nikosaliagas                    -1.8192      0.118    -15.464      0.000      -2.050      -1.589
dummy_politicalfigures                 0.6753      0.073      9.259      0.000       0.532       0.818
dummy_soccer                           0.6963      0.155      4.503      0.000       0.393       0.999
dummy_sportsfitnessbusiness            0.3082      0.143      2.148      0.032       0.027       0.589
dummy_tweet_period_afternoon          -0.7250      0.050    -14.603      0.000      -0.822      -0.628
dummy_tweet_period_morning            -0.7320      0.051    -14.292      0.000      -0.832      -0.632
dummy_tweet_period_night              -0.7384      0.048    -15.462      0.000      -0.832      -0.645
hashtags_count                         0.1799      0.036      5.001      0.000       0.109       0.250
mentions_count                        -0.9379      0.029    -32.293      0.000      -0.995      -0.881
tweet_external_urls_count              0.8408      0.053     15.762      0.000       0.736       0.945
tweet_length                           0.4895      0.022     21.977      0.000       0.446       0.533
tweet_medias_count                     0.6849      0.044     15.394      0.000       0.598       0.772
followers_count                        0.8017      0.013     60.866      0.000       0.776       0.828
==============================================================================
Omnibus:                      175.389   Durbin-Watson:                   1.411
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              296.625
Skew:                           0.218   Prob(JB):                     3.88e-65
Kurtosis:                       3.898   Cond. No.                     6.41e+15
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 3.23e-26. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""